In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gt_parser
import os

# root = "/home/ysheng/Dataset/soft_shadow/train/"
# sub_folder = "notsimulated_combine_male_short_outfits_genesis8_armani_casualoutfit03_Base_Pose_Standing_A"
# out_folder = "/home/ysheng/Dataset/soft_shadow/single_human/"
# test_path = os.path.join(root, os.path.join(sub_folder, "ground_truth.txt"))

# gt_dict = gt_parser.parse(test_path)
# print(len(gt_dict))

In [3]:
dataset_folder = '/home/ysheng/Dataset/soft_shadow/train'

out_file = os.path.join(dataset_folder, "metadata.csv")

gt_parser.parse_folder(dataset_folder, out_file)

cannot find file  /home/ysheng/Dataset/soft_shadow/train/.ipynb_checkpoints/ground_truth.txt


In [4]:
import csv
import numpy as np
import os
from shadow_render import render_shadow
import matplotlib.pyplot as plt
from tqdm import tqdm

"""input: x_y_z
   output: np array
"""
def get_vector(vec_str):
    x_ind = vec_str.find('_')
    x = float(vec_str[:x_ind])
    y_ind = vec_str[x_ind + 1:].find('_')
    
    x = float(vec_str[:x_ind])
    y = float(vec_str[x_ind + 1:x_ind + 1 + y_ind])
    z = float(vec_str[x_ind + 1 + y_ind + 1:])
    return np.array([x,y,z])

with open(out_file) as f:
    csv_read = csv.reader(f, delimiter=',')
    for r in tqdm(csv_read):
        mask_path = r[1]
        relative_vec_str = r[-1]
        relative_vec = get_vector(relative_vec_str)
        
        folder = os.path.dirname(mask_path)
        prefix = os.path.basename(mask_path)[:os.path.basename(mask_path).find("_")]
        
        light_path = os.path.join(folder,"{}_light.png".format(prefix))
        
#         print(folder)
#         print(light_path)
        img = render_shadow(relative_vec)
        plt.imsave(light_path,img)


80352it [16:04, 83.32it/s] 


In [ ]:
import matplotlib.pyplot as plt

dir_folder = os.path.join(out_folder, sub_folder)
test_file = os.path.join(dir_folder,"00004799_final.png")


img = plt.imread(test_file)
plt.figure()
plt.imshow(img)
plt.show()

In [ ]:
from shadow_render import render_shadow
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage.filters import gaussian_filter

def show(img):
    img = gaussian_filter(img, 2)
    plt.figure()
    plt.imshow(img)

testing_vec = np.array([0.0, 0.0, -1.0])
img = render_shadow(testing_vec)
show(img)

testing_vec = np.array([0.0, 0.0, 1.0])
img = render_shadow(testing_vec)
show(img)

testing_vec = np.array([0.0, 1.0, 0.0])
img = render_shadow(testing_vec)
show(img)

testing_vec = np.array([0.0, -1.0, 0.0])
img = render_shadow(testing_vec)
show(img)

testing_vec = np.array([1.0, 0.0, 0.0])
img = render_shadow(testing_vec)
show(img)

testing_vec = np.array([-1.0, 0.0, 0.0])
img = render_shadow(testing_vec)
show(img)
